# [Размер случайного леса](https://www.coursera.org/learn/vvedenie-mashinnoe-obuchenie/programming/57ezh/razmier-sluchainogho-liesa)

## Введение
Случайный лес — это модель классификации, объединяющая некоторое количество решающих деревьев в одну композицию, за счет чего улучшается их качество работы и обобщающая способность. Деревья строятся независимо друг от друга. Чтобы они отличались друг от друга, обучение проводится не на всей обучающей выборке, а на ее случайном подмножестве. Также, для дальнейшего уменьшения схожести деревьев, оптимальный признак для разбиения выбирается не из всех возможных признаков, а лишь из их случайного подмножества. Прогнозы, выданные деревьями, объединяются в один ответ путем усреднения.

Особенность случайного леса заключается в том, что он не переобучается по мере увеличения количества деревьев в композиции. Это достигается за счет того, что деревья не зависят друг от друга, и поэтому добавление нового дерева в композицию не усложняет модель, а лишь понижает уровень шума в прогнозах.

---

В библиотеке scikit-learn случайные леса реализованы в классах sklearn.ensemble.RandomForestClassifier (для классификации) и sklearn.ensemble.RandomForestRegressor (для регрессии). Обучение модели производится с помощью функции fit, построение прогнозов — с помощью функции predict. Число деревьев задается с помощью поля класса n_estimators. 

## 1. Загрузите данные из файла abalone.csv. 
Это датасет, в котором требуется предсказать возраст ракушки (число колец) по физическим измерениям.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor

In [2]:
data = pd.read_csv('./data/abalone.csv')
data.head()

,Sex,Length,Diameter,Height,WholeWeight,ShuckedWeight,VisceraWeight,ShellWeight,Rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


## 2. Преобразуйте признак Sex в числовой: значение F должно перейти в -1, I — в 0, M — в 1. 
Если вы используете Pandas, то подойдет следующий код:  
`data['Sex'] = data['Sex'].map(lambda x: 1 if x == 'M' else (-1 if x == 'F' else 0))`

In [3]:
data['Sex'] = data['Sex'].map(lambda x: 1 if x == 'M' else (-1 if x == 'F' else 0))
data.head()

,Sex,Length,Diameter,Height,WholeWeight,ShuckedWeight,VisceraWeight,ShellWeight,Rings
0,1,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,-1,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,1,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,0,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


## 3. Разделите содержимое файлов на признаки и целевую переменную. 
В последнем столбце записана целевая переменная, в остальных — признаки.

In [4]:
X = data.loc[:, :'ShellWeight']
y = data['Rings']

## 4. Обучите случайный лес (sklearn.ensemble.RandomForestRegressor) с различным числом деревьев: от 1 до 50 (не забудьте выставить "random_state=1" в конструкторе). 
Для каждого из вариантов оцените качество работы полученного леса на кросс-валидации по 5 блокам. Используйте параметры "random_state=1" и "shuffle=True" при создании генератора кросс-валидации sklearn.cross_validation.KFold. В качестве меры качества воспользуйтесь коэффициентом детерминации (sklearn.metrics.r2_score).

In [5]:
# clf = RandomForestRegressor(random_state=1)
clf = RandomForestRegressor()
cv = KFold(n_splits=5, shuffle=True, random_state=1)
grid = {'n_estimators': np.arange(1, 51)}
gs = GridSearchCV(clf, grid, scoring='r2', cv=cv, n_jobs=-1)
gs.fit(X, y)

GridSearchCV(cv=KFold(n_splits=5, random_state=1, shuffle=True),
       error_score='raise-deprecating',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'n_estimators': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='r2', verbose=0)

## 5. Определите, при каком минимальном количестве деревьев случайный лес показывает качество на кросс-валидации выше 0.52. Это количество и будет ответом на задание.

In [6]:
for k, v in gs.cv_results_.items():
    if 'mean' in k:
        print(f"Key: {k}, value: {v}\n")

Key: mean_fit_time, value: [0.01953421 0.03459702 0.05177283 0.06487432 0.08048673 0.09579158
 0.11161208 0.12725248 0.14249964 0.15787835 0.17301269 0.18859067
 0.20486484 0.21954017 0.24406734 0.25132675 0.26570926 0.28294001
 0.29817734 0.31409006 0.32814393 0.34266253 0.35968566 0.37774596
 0.38928671 0.40389643 0.42044344 0.43914847 0.4528646  0.46698937
 0.48403192 0.49797921 0.5129365  0.5303956  0.5451755  0.56157756
 0.57486916 0.5913681  0.60691714 0.62254338 0.63696299 0.65322599
 0.66758509 0.68523769 0.69946551 0.71447802 0.73379922 0.74514804
 0.76323934 0.75723276]

Key: mean_score_time, value: [0.00214529 0.00228462 0.00246477 0.00268607 0.0029161  0.0032043
 0.0033823  0.0036077  0.00391598 0.00413151 0.00435128 0.00458846
 0.00482397 0.00507789 0.00532532 0.00555868 0.00581756 0.00613308
 0.00631065 0.00654726 0.0067976  0.00704756 0.00729589 0.00755434
 0.00778489 0.00800982 0.00830059 0.00852633 0.00878716 0.00903502
 0.00927563 0.00949655 0.00976491 0.01001153 0.01

In [14]:
for i, val in enumerate(gs.cv_results_['mean_test_score']):
    if val > 0.52:
        print(grid['n_estimators'][i])
        break

22
